<a href="https://colab.research.google.com/github/h1lla7y/Projects/blob/main/monte_carlo_sofr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests

In [2]:
avg_loan = 25000000
num_loans = 15
sofr_spread = .05

np.random.seed(42)
portfolio = pd.DataFrame({
    'loan_id': range(1, num_loans + 1 ),
    'amount': np.random.lognormal(mean=np.log(avg_loan), sigma=.05, size= num_loans),
    'ltv': np.random.uniform(0.5, 0.8, size=num_loans),
    'probd': np.random.uniform(0.01, 0.10, size=num_loans)
})


In [4]:
try:
    sofr_rates = pd.read_csv("sofr.csv", parse_dates=['Effective Date'], index_col='Effective Date')['Rate (%)'] / 100
    sofr_rates = sofr_rates.dropna()
    assert len(sofr_rates) > 0, "Data empty after loading"
except Exception as e:
    raise ValueError(f"Failed to load SOFR data: {e}")


sofr_returns = np.log(sofr_rates / sofr_rates.shift(1)).dropna()
mu_sofr = sofr_returns.mean() * 252
sigma_sofr = sofr_returns.std() * np.sqrt(252)


def simulate_sofr_paths(initial_rate, mu, sigma, days=252, n_simulations=1000):
    dt = 1/days
    paths = np.zeros((days, n_simulations))
    paths[0] = initial_rate

    for t in range(1, days):
        shocks = np.random.normal(0, 1, n_simulations)
        paths[t] = paths[t-1] * np.exp((mu - 0.5*sigma**2)*dt + sigma*np.sqrt(dt)*shocks)
    return paths


initial_sofr = .045
new_sofr_paths = simulate_sofr_paths(initial_sofr, mu_sofr, sigma_sofr)
print("Simulation successful!")

Simulation successful!


In [5]:
n_simulations = 1000
recovery_rate = 0.85
losses = np.zeros(n_simulations)
avg_sofr_per_sim = np.zeros(n_simulations)


for i in range(n_simulations):
    sofr_horizon = new_sofr_paths[-1, i]
    avg_sofr_per_sim[i] = np.mean(new_sofr_paths[:, i])


    probd_adjusted = portfolio['probd'] * (1 + 2*(sofr_horizon - initial_sofr)/initial_sofr)
    probd_adjusted = np.clip(probd_adjusted, 0.01, 0.30)

    ltv_adjusted = portfolio['ltv'] * (1 + 0.1*(sofr_horizon - initial_sofr)/initial_sofr)

    defaults = np.random.binomial(1, probd_adjusted, size=num_loans)
    losses[i] = np.sum(portfolio['amount'] * (1-recovery_rate) * defaults * (ltv_adjusted > 0.8))



In [6]:
var_95 = np.percentile(losses, 95)
expected_loss = np.mean(losses)

print(f" Expected Annual Loss: {expected_loss:,.3f}")


 Expected Annual Loss: 422,018.080


In [7]:
print(f"SOFR stats - Mean: {np.mean(new_sofr_paths):.3f}, Std: {np.std(new_sofr_paths):.3f}")
print(f"Loss stats - Mean: ${np.mean(losses):,.3f}, Std: ${np.std(losses):,.3f}")

SOFR stats - Mean: 0.038, Std: 0.141
Loss stats - Mean: $422,018.080, Std: $2,426,903.349
